# 3. 装饰器

类似于程序设计语义学中所说的前键 与后键 的概念(Eiffel中的@pre 与@post )。当然你可以在装饰器中做比前键与后键更多的事，如：引入日志、增加计时逻辑来检测性能、给函数增加事务的能力。

 
其实总体说起来，装饰器其实也就是一个函数，一个用来包装函数的函数，返回一个修改之后的函数对象，被装饰函数感受不到装饰器的存在.

**原则：** 

    不能修改被装饰的函数的源代码（比如线上环境）
    不能修改被装饰的函数的调用方式 
    
**功能**

    装饰其他函数，就是为其他函数添加附加功能   

## 3.1 实现装饰器知识储备： 

    函数即是“变量”
    高阶函数
    嵌套函数
    
    高阶函数+嵌套函数=>装饰器

先分析以下两段代码能不能运行？

In [ ]:
def foo1():
    print("in the foo")
    bar1()
    
def bar1():
    print("in the bar")

foo1()

In [ ]:
def foo2():
    print("in the foo")
    bar2()

foo2()
def bar2():
    print("in the bar")

第二段代码报错：

    NameError: name 'bar2' is not defined

**定义变量：**

如：定义变量：x=1,会在内存中找块内存空间把“1”存进去，把“1”的内存地址给x 

**函数即变量**

In [ ]:
# 定义函数
def test():
    pass

# 就相当于把函数体赋值给test变量
test = '函数体'  # 函数体就是一堆字符串而已
# 只不过函数调用要加上小括号调用
test()

**python内存回收机制**

    解释器到底怎么去回收这个变量？ 
    python解释器当中有种概念叫做引用计数。什么叫引用计数呢？ 
    比如：定义x=1，之后又定义了y=1或y=x，实际上又把内存空间“1”的内存地址赋值给y 
    这里x代表一次引用，y代表一次引用。加起来两次引用。 
    python什么时候会把“1”这个内存空间清空呢？会回收内存呢？ 
    当x这个变量没有了，y这个变量也没有了，便会把“1”这个内存空间清掉

    del x  # 删的只是变量名，内存中的值是解释器回收

**匿名函数**

In [ ]:

lambda x:x*x

匿名函数没有函数名，没有引用，所以会被垃圾回收机制立马回收掉。 
所以匿名函数要赋值给变量，把函数体赋值给变量名

calc = lambda x:x*x
print(calc(4))

现在可以再理解下最开始两段代码能不能运行的原因。 

### 3.1.1 高阶函数(装饰器前奏) 

什么叫高阶函数呢：

    把一个函数名当做形实传给另外一个函数
    返回值中包含函数名

In [19]:
def f1():
    print("in the func1")

def test1(func):
    print(func)

test1(f1)  # 运行结果(打印内存地址)

<function f1 at 0x000001AA1ACA4158>


In [20]:
# 如下代码，能不能运行：
def f1():
    print("in the func1")

def test1(func):
    print(func)
    func()

test1(f1)

<function f1 at 0x000001AA1ACA4400>
in the func1


函数即变量，像“x=1,y=x”，同样f是一个是一个函数，可不可以像一个变量一样来回赋值呢？

In [21]:
import time
def func1():
    print("in the func1")
    time.sleep(1)
    
def test1(func):
    start_time = time.time()
    func()
    stop_time = time.time()
    print("the func run time is %s" %(stop_time-start_time))

test1(func1)

in the func1
the func run time is 1.0006282329559326


    到这里，貌似实现了装饰函数的功能。 
    看上面装饰器的原则： 
    这里：没有修改func1的源代码，但是调用方式改变了。现在是test1(func1)，之前是func1() 
    现在能做到哪一点呢？ 
    
    把一个函数名当做实参传给另外一个函数（不修改被装饰的函数源代码的情况下为其添加功能）

下面用第二个条件（返回值中包含函数名），做另外一个高阶函数

In [22]:
import time

def func2():
    time.sleep(1)
    print("in the func2")
def test2(func):
    print(func)
    return(func)

print(test2(func2))

<function func2 at 0x000001AA1ABC7950>
<function func2 at 0x000001AA1ABC7950>


把函数内存地址都打印出来了，看到这么多内存地址，有什么想法？ 

加上小括号就能运行。 

上面代码“test2(func2())”和“test2(func2)”有什么区别？加上小括号是函数返回结果，不加是函数内存地址。所以加上小括号就不符合高阶函数定义了。 

既然以后有了函数的内存地址，是不是可以赋值给其他变量？

In [24]:
import time

def func2():
    print("in the func2")
    time.sleep(1)
    
def test2(func):
    print(func)
    return(func)

func2 = (test2(func2))
func2()

<function func2 at 0x000001AA1ACBCD90>
in the func2


这就是高阶函数的第二个好处：返回值中包含函数名（不修改函数的调用方式）

### 3.1.2 嵌套函数（装饰器前戏）

嵌套函数：在一个函数体内，用def去声明一个函数

In [25]:
def foo():
    print("in the foo")
    def bar():
        print("in the bar")
    bar()

foo()

in the foo
in the bar


In [ ]:
注意 函数嵌套和函数调用是不同的

In [ ]:
def foo():
    print("in the foo")
def bar():
    foo()

bar()

In [ ]:
局部作用域和全局作用域的访问顺序：

In [26]:
x = 0
def grandpa():
    # x = 1
    def dad():
        x = 2
        def son():
            x = 3
            print(x)
        son()
    dad()
    
grandpa()

3


## 3.2 装饰器实例

In [29]:
import time

def deco(func):
    start_time = time.time()
    func()
    stop_time = time.time()
    print("the func tun time is %s" %(stop_time-start_time))
    
def test1():
    time.sleep(1)
    print("in the test1")
          
def test2():
    time.sleep(1)
    print("in the test2")

          
deco(test1)
deco(test2)

in the test1
the func tun time is 1.000476360321045
in the test2
the func tun time is 1.000396728515625


In [ ]:
按照上面说的，如何实现不改变调用方式？直接“test1 = deco(test1)”和“test2 = deco(test2)”吗？ 
别忘记了，第二种方式，高阶函数要加上return，如下

In [2]:
import time

def deco(func):
    start_time = time.time()
    return func()
    stop_time = time.time()
    print("the func tun time is %s" %(stop_time-start_time))
    
def test1():
    time.sleep(1)
    print("in the test1")
    
def test2():
    time.sleep(1)
    print("in the test2")

    
test1 = deco(test1)
test2 = deco(test2)

deco(test1)
deco(test2)

in the test1
in the test2


TypeError: 'NoneType' object is not callable

虽然没有修改源代码和调用方式，但是函数加上return，函数就结束了，然并卵。怎么实现呢？ 
前面一直在用高阶函数，还没有用嵌套函数，加上嵌套函数能不能实现呢？看一下

In [5]:
import time

def timer(func):  # timer(test1)  func=test1
    def deco():
        start_time = time.time()
        func()
        stop_time = time.time()
        print("the func tun time is %s" %(stop_time-start_time))
    return deco  # 返回deco的内存地址

def test1():
    time.sleep(1)
    print("in the test1")
    
def test2():
    time.sleep(1)
    print("in the test2")

print(timer(test1))  # 可见：返回deco的内存地址
test1 = timer(test1)
test1()
timer(test2)()

<function timer.<locals>.deco at 0x105ee4b70>
in the test1
the func tun time is 1.0053858757019043
in the test2
the func tun time is 1.0036380290985107


到此，完成实现了装饰器的功能。但是还是有点麻烦，如何能不要“test1 = timer(test1)”， 
python解释器提供了语法糖“@”符合，给哪个函数新增功能，就加在哪个函数头部

In [32]:
import time

def timer(func):  # timer(test1)  func=test1
    def deco():
        start_time = time.time()
        func()
        stop_time = time.time()
        print("the func tun time is %s" %(stop_time-start_time))
    return deco  # 返回deco的内存地址

@timer
def test1():
    time.sleep(1)
    print("in the test1")
    
@timer
def test2():
    time.sleep(1)
    print("in the test2")

test1()
test2()

in the test1
the func tun time is 1.0004403591156006
in the test2
the func tun time is 1.0002806186676025


## 3.3 装饰器语法

### 3.1.1 无参数装饰器

In [6]:
def deco(func):  
    print(func)  
    return(func)  

@deco  
def foo():
    pass

foo()  

<function foo at 0x105e52bf8>


第一个函数deco是装饰函数，它的参数就是被装饰的函数对象。我们可以在deco函数内对传入的函数对象做一番“装饰”，然后返回这个对象（记住一定要返回 ，不然外面调用foo的地方将会无函数可用。实际上此时foo=deco(foo)）

### 3.1.2 写了个小例子，检查函数有没有说明文档：

In [7]:
def deco_functionNeedDoc(func):  
    if func.__doc__ == None :  
        print(func, "has no __doc__, it's a bad habit.")
    else:  
        print(func, ':', func.__doc__, '.')  
    return func  

@deco_functionNeedDoc  
def f():  
    print('f() Do something')
    
@deco_functionNeedDoc  
def g():  
    'I have a __doc__'  
    print('g() Do something')
    

f()  
g()  

<function f at 0x105ee81e0> has no __doc__, it's a bad habit.
<function g at 0x105ee88c8> : I have a __doc__ .
f() Do something
g() Do something


## 3.3.3 有参数装饰器

前面实现了装饰器的功能，但是如果函数有参数，能不能也能运行呢

In [9]:
def decomaker(arg):  
    '通常对arg会有一定的要求'  
    """由于有参数的decorator函数在调用时只会使用应用时的参数  
       而不接收被装饰的函数做为参数，所以必须在其内部再创建  
       一个函数  
    """  
    def newDeco(func):    #定义一个新的decorator函数  
        print(func, arg)  
        return(func)  
    return newDeco  

@decomaker
def foo(name):
    pass

foo("python")

python <function foo at 0x105ef98c8>


'python'

第一个函数decomaker是装饰函数，它的参数是用来加强“加强装饰”的。由于此函数并非被装饰的函数对象，所以在内部必须至少创建一个接受被装饰函数的函数，然后返回这个对象（实际上此时foo=decomaker(arg)(foo)）

一个错误的例子

In [8]:
import time
def timer(func):  # timer(test1)  func=test1
    def deco():
        start_time = time.time()
        func()
        stop_time = time.time()
        print("the func tun time is %s" %(stop_time-start_time))
    return deco  # 返回deco的内存地址

@timer
def test1():
    time.sleep(1)
    print("in the test1")
    
@timer
def test2(name):
    time.sleep(1)
    print("in the test2",name)

test1()
test2('python')

in the test1
the func tun time is 1.0035510063171387


TypeError: deco() takes 0 positional arguments but 1 was given

修改代码 

    @timer 相当于 test2=timer(test2) =deco 
    test2() 相当于运行deco()，所以没指定参数，报错。 
    如何传参数呢？为了适应各种不同参数的函数

In [40]:
import time
def timer(func):  # timer(test1)  func=test1
    def deco(*args,**kwargs):
        start_time = time.time()
        func(*args,**kwargs)
        stop_time = time.time()
        print("the func tun time is %s" %(stop_time-start_time))
    return deco  # 返回deco的内存地址

@timer
def test1():
    time.sleep(1)
    print("in the test1")
    
@timer
def test2(name):
    time.sleep(1)
    print("in the test2",name)

test1()
test2("fgf")

in the test1
the func tun time is 1.0002033710479736
in the test2 fgf
the func tun time is 1.0015861988067627


## 3.4 终极装饰器

注意，上面的例子中还没有涉及返回值，看下面的例子可以体会一下 
假设：公司网站需要验证登录，有不同的验证方式：本地认证、LDAP认证等

In [43]:
#/usr/bin/env python
# -*- coding: UTF-8 -*-

import time
user,passwd = 'fgf','abc123'
def auth(auth_type):  #装饰器参数
    #print("auth func:",auth_type)
    def outer_wrapper(func):  #被装饰的函数
        def wrapper(*args, **kwargs):
            print("wrapper func args:", *args, **kwargs) #函数参数，无
            if auth_type == "local":
                username = input("Username:").strip()
                password = input("Password:").strip()
                if user == username and passwd == password:
                    print("\033[32;1mUser has passed authentication\033[0m")
                    res = func(*args, **kwargs)  # from home
                    print("---after authenticaion ")
                    return res
                else:
                    print("\033[31;1mInvalid username or password\033[0m")
                    exit("\033[31;1mInvalid username or password\033[0m")
            elif auth_type == "ldap":
                print("搞毛线ldap,不会。。。。")

        return wrapper
    return outer_wrapper

def index():
    print("welcome to index page")
    
@auth(auth_type="local") # home = wrapper()
def home():
    print("welcome to home  page")
    return "from home"

@auth(auth_type="ldap")
def bbs():
    print("welcome to bbs  page")

index()
print(home()) #wrapper()
#bbs()

welcome to index page
wrapper func args:
Username:ddd
Password:fefew
Invalid username or password
None


最简单的模板是这样的

In [26]:
#-*-coding:utf-8-*-

def outer(func):
    def inner():
        print('before')
        func()
        print('after')
        # return r
    return inner
'''
def outer_1(func):
    def inner():
        print 'before 11'
        func()
        print 'after 22'
        # return r
    return inner

@outer_1
'''
@outer
def F1():
    print('test')


F1()


before
test
after


函数带多个参数，装饰器对应修改以适合多种情

In [28]:
# -*- coding: utf-8 -*-

from time import ctime
from time import sleep

def ftfunc(func):
    def timef(*s,**gs):
        print("[%s] %s() called" % (ctime(),func.__name__))
        return func(*s,**gs)
    return timef

@ftfunc
def foo(*s,**gs):
    print(s)
    print(gs)

if __name__ == '__main__':
    foo()
    foo(1)
    foo(1,2)
    foo(1,2,3)
    stu = {'name' : 'alam' ,'age' :12}
    foo(1,2,3,**stu)


[Sat Apr 21 23:17:29 2018] foo() called
()
{}
[Sat Apr 21 23:17:29 2018] foo() called
(1,)
{}
[Sat Apr 21 23:17:29 2018] foo() called
(1, 2)
{}
[Sat Apr 21 23:17:29 2018] foo() called
(1, 2, 3)
{}
[Sat Apr 21 23:17:29 2018] foo() called
(1, 2, 3)
{'name': 'alam', 'age': 12}


函数带多个参数，装饰器也带多个参数

In [37]:
# -*- coding: utf-8 -*-

def decrator(*dargs, **dkargs):   #装饰器参数
    print("decrator param:", dargs, dkargs)  
    def wrapper(func):
        def _wrapper(*args, **kargs):
            print("decrator param:", dargs, dkargs) 
            print("function param:", args, kargs)
            return func(*args, **kargs)
        return _wrapper
    return wrapper

@decrator(1, a=2)  #执行装饰器， 此时 func为None
def foo(x, y=0):
    print("foo", x, y)


decrator param: (1,) {'a': 2}


In [38]:
foo(3, y=4)

decrator param: (1,) {'a': 2}
function param: (3,) {'y': 4}
foo 3 4


函数带多个参数，装饰器能转换参数类型

In [41]:
# -*- coding: utf-8 -*-

def validate(**vkargs): #  装饰器参数
    def decorator(func): #   被装饰的函数
        def wrapper(**kargs): #函数参数
            print('v', vkargs)
            print('k', kargs)
            for key in vkargs: 
                # 根据vkargs中的参数的类型对kargs的参数进行类型转换
                kargs[key] = vkargs[key](kargs[key])
            return func(**kargs)
        return wrapper
    return decorator


@validate(x=int, y=float, z=float)
def move(x, y, z):
    print("move %d (%0.2f, %0.2f)"%(x, y, z))



if __name__ == '__main__':
    move(x="1", y="0.2", z="0.3")

v {'x': <class 'int'>, 'y': <class 'float'>, 'z': <class 'float'>}
k {'x': '1', 'y': '0.2', 'z': '0.3'}
move 1 (0.20, 0.30)
